# Project-1 BETA TEAM

In [1]:
# Dependencies
import pandas as pd
import numpy as np
from numpy import NaN
import matplotlib.pyplot as plt


In [2]:
# Extracting Data from CSV to Jupyter-Notebook
prices_file = "Resources/prices.csv"
places_file = "Resources/places.csv"
prices_df = pd.read_csv(prices_file)
places_df = pd.read_csv(places_file)

In [3]:
# Cleaning Columns on Prices DataFrame
prices_df = prices_df.drop('places', axis = 1)
prices_df = prices_df.drop('place', axis = 1)
prices_df

,place_id,gas_price,type
0,11703.0,18.69,regular
1,NaN,19.69,premium
2,11702.0,18.19,regular
3,NaN,19.79,premium
4,NaN,18.25,diesel
...,...,...,...
33210,NaN,19.59,premium
33211,NaN,18.60,diesel
33212,2807.0,17.99,regular
33213,2807.0,19.99,premium


In [5]:
# Creating a column to replace NaNs with Places_ID
Place_ID = prices_df['place_id'].fillna(method='ffill')
# Adding Column to existing prices DataFrame
prices_df['Place_ID'] = Place_ID
# Eliminating original place_id from DataFrame
prices_df = prices_df.drop('place_id', axis = 1)
# Rearrange DataFrame
prices_df = prices_df[['Place_ID','gas_price','type']]
# Rename Place_ID
prices_df.columns = ['_place_id', 'gas_price','type']
prices_df

KeyError: 'place_id'

In [6]:
# Merging DataFrames
Gas_Stations_df = pd.merge(places_df, prices_df, on="_place_id")
Gas_Stations_df

,name,cre_id,location/x,location/y,_place_id,gas_price,type
0,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.92140,32.47641,2039,17.39,regular
1,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.92140,32.47641,2039,19.35,premium
2,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.92140,32.47641,2039,18.45,diesel
3,"DIGEPE, S.A. DE C.V. (07356)",PL/902/EXP/ES/2015,-99.74484,20.30370,2040,18.65,regular
4,"DIGEPE, S.A. DE C.V. (07356)",PL/902/EXP/ES/2015,-99.74484,20.30370,2040,18.99,premium
...,...,...,...,...,...,...,...
33210,GRUPO GASOLINERO DEL DIAMANTE HOLZ SA DE CV,PL/23508/EXP/ES/2020,-103.40990,22.83751,27300,18.59,premium
33211,GRUPO GASOLINERO DEL DIAMANTE HOLZ SA DE CV,PL/23508/EXP/ES/2020,-103.40990,22.83751,27300,18.89,diesel
33212,JAGREGAS 2 S.A. DE C.V.,PL/23510/EXP/ES/2020,-100.41290,20.67198,27304,16.99,regular
33213,JAGREGAS 2 S.A. DE C.V.,PL/23510/EXP/ES/2020,-100.41290,20.67198,27304,17.30,premium


In [7]:
# Process to get complementary information from google maps API
import requests
import json

# Google developer API key
from config import gkey


In [77]:
# Preparing API Call from google using iterrow() method
original = pd.read_csv("../Project-1/Resources/Places.csv")

# Let's try with variables relevants for the call
Gas_Stations = original.loc[:, ["name", "cre_id", "location/x", "location/y"]]

# Add a "Hotel Name" column to the DataFrame.
Gas_Stations["rating"] = ""
Gas_Stations.rename(columns={"name":"Name", "cre_id":"ID", "location/x":"Lng", "location/y":"Lat", "rating":"Rating"}, inplace=True)
Gas_Stations
# Getting a subset to proof
Gas_Stations_sub = Gas_Stations.head()
Gas_Stations_sub

,Name,ID,Lng,Lat,Rating
0,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.92140,32.47641,
1,"DIGEPE, S.A. DE C.V. (07356)",PL/902/EXP/ES/2015,-99.74484,20.30370,
2,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,-106.45140,31.71947,
3,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,-103.30420,20.71413,
4,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,-98.29977,26.03787,


In [78]:
rating = Gas_Stations_sub.loc[:,['Name', 'Lat', 'Lng', 'Rating']]
rating = pd.DataFrame(rating)
rating

,Name,Lat,Lng,Rating
0,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",32.47641,-116.92140,
1,"DIGEPE, S.A. DE C.V. (07356)",20.30370,-99.74484,
2,"DIAZ GAS, S.A. DE C.V.",31.71947,-106.45140,
3,"COMBU-EXPRESS, S.A. DE C.V.",20.71413,-103.30420,
4,"PETROMAX, S.A. DE C.V.",26.03787,-98.29977,


In [126]:
params = {
    "rankby":"distance",
    "type": "gas_station",
    "keyword": "fuel station",
    #"radius": 100,
    "key": gkey    
}

# Hit the Google Places API for each city's coordinates.
url_base = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [127]:
# Store the first Hotel result into the DataFrame.
for index, row in rating.head(1).iterrows():
    lat=row["Lat"]
    lon=row["Lng"]
        
    params["location"] = f"{lat},{lon}"
    
    print(f"Results for observation: {index}")
    
    response = requests.get(url_base, params=params).json()
# To storage the requests

    results = response['results']

    
    

Results for observation: 0


In [129]:
A = json.dumps(response, indent=4, sort_keys=True)
# with open("gas_stations_json", 'w') as json_file:
#    json_dump(results, json_file)

In [134]:
response["results"][0] #validación if 

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 32.4764095, 'lng': -116.9213067},
  'viewport': {'northeast': {'lat': 32.47781157989272,
    'lng': -116.9198421701073},
   'southwest': {'lat': 32.47511192010728, 'lng': -116.9225418298928}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/gas_station-71.png',
 'name': 'Gasolinera Pemex',
 'place_id': 'ChIJVVWV7x052YARndH9miROVUk',
 'plus_code': {'compound_code': 'F3GH+HF Tijuana, Baja California',
  'global_code': '8545F3GH+HF'},
 'rating': 4.3,
 'reference': 'ChIJVVWV7x052YARndH9miROVUk',
 'scope': 'GOOGLE',
 'types': ['gas_station', 'point_of_interest', 'establishment'],
 'user_ratings_total': 3,
 'vicinity': 'Av de los Insurgentes No. 1910, Libramiento, Tijuana'}

In [112]:
# Paths to get the info in json responses

response["results"][0]["geometry"]["location"]["lat"] 
response["results"][0]["geometry"]["location"]["lng"]
response["results"][0]["rating"]
response["results"][0]["name"]
response["results"][0]["user_ratings_total"]
response["results"][0]["vicinity"]
response["results"][0]["types"]



['gas_station', 'point_of_interest', 'establishment']

In [104]:
response["results"][0]["rating"]


4.6

In [111]:
rating ["lat"] =""
rating["lng"] = ""
rating["name"] = ""
rating["user_rating_total"] = ""
rating["types"] = ""
rating["vicinity"]= ""
rating


,Name,Lat,Lng,Rating,lat,lng,name,user_rating_total,types,vicinity
0,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",32.47641,-116.92140,,,,,,,
1,"DIGEPE, S.A. DE C.V. (07356)",20.30370,-99.74484,,,,,,,
2,"DIAZ GAS, S.A. DE C.V.",31.71947,-106.45140,,,,,,,
3,"COMBU-EXPRESS, S.A. DE C.V.",20.71413,-103.30420,,,,,,,
4,"PETROMAX, S.A. DE C.V.",26.03787,-98.29977,,,,,,,
